<a href="https://colab.research.google.com/github/shubhanshu-26/Big_data/blob/main/IMDB_Movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz -O spark.tgz
!tar -xzf spark.tgz

--2025-04-14 13:43:36--  https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400395283 (382M) [application/x-gzip]
Saving to: ‘spark.tgz’

spark.tgz           100%[===================>] 381.85M   837KB/s    in 11m 31s 

2025-04-14 13:55:07 (566 KB/s) - ‘spark.tgz’ saved [400395283/400395283]



In [3]:
!pip install pyspark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("PySpark Clean Setup") \
    .getOrCreate()

In [6]:
df = spark.createDataFrame([
    ("Alice", 23),
    ("Bob", 34),
    ("Charlie", 29)
], ["Name", "Age"])

df.show()

+-------+---+
|   Name|Age|
+-------+---+
|  Alice| 23|
|    Bob| 34|
|Charlie| 29|
+-------+---+



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
movies_path = "/content/drive/MyDrive/Spark-datasets/movies.csv"
ratings_path = "/content/drive/MyDrive/Spark-datasets/ratings.csv"

moviesdf = spark.read.option("header", "true").option("inferSchema", "true").csv(movies_path)
ratingsdf = spark.read.option("header", "true").option("inferSchema", "true").csv(ratings_path)

moviesdf.show(5)
ratingsdf.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
+------+-------+------+----------+
only showing top 5 rows



In [5]:
# Now running IMDB query(# Which IMDB movie has the most number of ratings)

# covert dataframe in which we were executing is DSL into normal table so that we can work with SQL
ratingsdf.createOrReplaceTempView("ratings")
moviesdf.createOrReplaceTempView("movies")

In [15]:
ratingsdf.printSchema()
moviesdf.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [16]:
spark.sql('''select title, count(*) as rating_count
             from movies m join ratings r ON m.movieId = r.movieId
             where genres = 'Comedy' group by title
             order by rating_count desc limit 1''').show()
# Now running IMDB query(# Which IMDB movie has the most number of ratings)

+--------------------+------------+
|               title|rating_count|
+--------------------+------------+
|Ace Ventura: Pet ...|       37453|
+--------------------+------------+



In [6]:
# show top-rated movies by genre with atleast 10 ratings
spark.sql('''
            SELECT m.genres, m.title, ROUND(AVG(r.rating), 2) as avg_rating, COUNT(*) as total_ratings
            FROM movies m
            JOIN ratings r ON m.movieId = r.movieId
            WHERE m.genres != '(no genres listed)'
            GROUP BY m.genres, m.title
            HAVING total_ratings >= 10
            ORDER BY m.genres, avg_rating DESC''').show(50, truncate=False)

+------+---------------------------------------------------------------------------------------+----------+-------------+
|genres|title                                                                                  |avg_rating|total_ratings|
+------+---------------------------------------------------------------------------------------+----------+-------------+
|Action|Baasha (1995)                                                                          |3.8       |10           |
|Action|Ip Man 2 (2010)                                                                        |3.78      |1209         |
|Action|Lone Wolf and Cub: Baby Cart in Peril (Kozure Ôkami: Oya no kokoro ko no kokoro) (1972)|3.72      |112          |
|Action|13 Assassins (Jûsan-nin no shikaku) (2010)                                             |3.7       |825          |
|Action|Ip Man 3 (2015)                                                                        |3.67      |367          |
|Action|The Beast Stalke

Github link:- https://github.com/shubhanshu-26/Big_data

In [ ]:
#  Save Recommendations to MySQL

top_movies_df = spark.sql("""
SELECT m.genres, m.title, ROUND(AVG(r.rating),2) as avg_rating, COUNT(*) as total_ratings
FROM movies_exploded m
JOIN ratings r ON m.movieId = r.movieId
GROUP BY m.genres, m.title
HAVING total_ratings >= 10
ORDER BY m.genres, avg_rating DESC
""")

top_movies_df.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/moviesdb") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "top_movies_by_genre") \
    .option("user", "youruser") \
    .option("password", "yourpassword") \
    .mode("overwrite") \
    .save()